In [1]:
from batchcreator import DataGenerator, get_list_IDs
from datetime import datetime
import tensorflow as tf
from model_builder import build_generator, GAN
import numpy as np

In [2]:
batch_size = 8

# Create list of IDs to retrieve
x_seq_size=5
y_seq_size=1

start_dt = datetime(2019,6,1,0,0)
end_dt =  datetime(2019,6,8,0,0)
# Exclude samples were there is no rain in the input
filter_no_rain = True
list_IDs = get_list_IDs(start_dt, end_dt, x_seq_size, y_seq_size, filter_no_rain=filter_no_rain)
print(list_IDs[0])
print(len(list_IDs))

(['201905312335', '201905312340', '201905312345', '201905312350', '201905312355'], ['201906010000'])
404


In [3]:
# Test generator
generator = DataGenerator(list_IDs, batch_size=batch_size, x_seq_size=x_seq_size, 
                                       y_seq_size=y_seq_size, normalize_y=False)
for i in generator:
    x,y = i
    print(x.shape, y.shape)

(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)
(32, 5, 768, 700, 1) (32, 1, 384, 350, 1)


In [3]:
model = GAN()
model.compile()

In [ ]:
generator = DataGenerator(list_IDs, batch_size=1, x_seq_size=x_seq_size, 
                                       y_seq_size=y_seq_size, normalize_y=False)
hist = model.fit(generator, epochs=1)

 14/404 [>.............................] - ETA: 1:33:14 - d_loss: 0.6904 - g_loss: 0.6949